# Data Cleansing
In this notebook, we will be cleaning the data and preparing it for analysis.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

from tqdm import tqdm

## Import the Data

In [15]:
# Import the data from csv files
rate_df = pd.read_csv('01_Raw_Data\/rate.csv', parse_dates=False)
stocks_df = pd.read_csv('01_Raw_Data\/stocks.csv', parse_dates=False)
dividends_df = pd.read_csv('01_Raw_Data\/dividends.csv', parse_dates=False)
options_df = pd.read_csv('01_Raw_Data\/options.csv', parse_dates=False)
options_prices_df = pd.read_csv('01_Raw_Data\/options_prices.csv', parse_dates=False)

## Create the Final DataFrame from the Options Prices DataFrame

In [16]:
# Create a new dataframe
df_final = options_prices_df
# Reset the index
df_final = df_final.reset_index()
# Drop the 'index' column
df_final = df_final.drop(columns=['index'])
# Rename the 'Date' column to 'date_original'
df_final = df_final.rename(columns={'Date': 'date_original'})

# Get the first 10 characters from the 'date_original' column
# and put them in a new column called 'Date'
df_final['Date'] = df_final['date_original'].str[:10]

# Drop the date_original column
df_final = df_final.drop(columns=['date_original'])

# Reorder the columns
df_final = df_final[['Date', 'Ticker', 'Close']]

df_final.head()

,Date,Ticker,Close
0,2022-08-25,AAPL230421C00050000,119.500000
1,2022-08-26,AAPL230421C00050000,117.349998
2,2022-08-29,AAPL230421C00050000,112.599998
3,2022-09-01,AAPL230421C00050000,107.599998
4,2022-09-02,AAPL230421C00050000,106.250000


## Extract the Stock Ticker from the Option Ticker

In [17]:
# Create a Stock Ticker Column
# extract all the letters from the string before the first number shows up
df_final['Stock Ticker'] = df_final['Ticker'].str.extract('([A-Z]+)')
df_final.head()

,Date,Ticker,Close,Stock Ticker
0,2022-08-25,AAPL230421C00050000,119.500000,AAPL
1,2022-08-26,AAPL230421C00050000,117.349998,AAPL
2,2022-08-29,AAPL230421C00050000,112.599998,AAPL
3,2022-09-01,AAPL230421C00050000,107.599998,AAPL
4,2022-09-02,AAPL230421C00050000,106.250000,AAPL


## Calculate the Days to Expiration from the Option Ticker

In [18]:
# Create Function to convert 230414 to 2023-04-14
def convert_date(date):
    date = str(date)
    year = '20' + date[0:2]
    month = date[2:4]
    day = date[4:6]
    return year + '-' + month + '-' + day

In [19]:
# Convert the 'Date' column to a datetime object
df_final['Date'] = pd.to_datetime(df_final['Date'])

In [20]:
# Extract the expiration date from the ticker
# extract all the numbers
df_final['Expiration Date'] = df_final['Ticker'].str.extract('(\d+)')
df_final['Expiration Date'] = df_final['Expiration Date'].apply(convert_date)
# Calculate Days to Expiration
df_final['Days to Expiration'] = (pd.to_datetime(df_final['Expiration Date']) - pd.to_datetime(df_final['Date'])).dt.days
# Drop the 'Expiration Date' column
df_final = df_final.drop(columns=['Expiration Date'])
df_final.head()

,Date,Ticker,Close,Stock Ticker,Days to Expiration
0,2022-08-25,AAPL230421C00050000,119.500000,AAPL,239
1,2022-08-26,AAPL230421C00050000,117.349998,AAPL,238
2,2022-08-29,AAPL230421C00050000,112.599998,AAPL,235
3,2022-09-01,AAPL230421C00050000,107.599998,AAPL,232
4,2022-09-02,AAPL230421C00050000,106.250000,AAPL,231


## Extract the Option Type from the Option Ticker

In [21]:
# Extract the option type from the ticker
df_final['Option Type'] = df_final['Ticker'].str.extract('[A-Za-z]+([0-9]+[A-Za-z])')
df_final['Option Type'] = df_final['Option Type'].apply(lambda x: x[-1])
# Create a dummy variable for the option type
df_final['Option Type'] = df_final['Option Type'].apply(
    lambda x: 1 if x == 'C' else 0)
# Rename the 'Option Type' column to 'Call'
df_final = df_final.rename(columns={'Option Type': 'Call'})
# Reorder the columns
df_final = df_final[['Date', 'Ticker', 'Call', 'Close', 'Stock Ticker',
                     'Days to Expiration']]
df_final.head()

,Date,Ticker,Call,Close,Stock Ticker,Days to Expiration
0,2022-08-25,AAPL230421C00050000,1,119.500000,AAPL,239
1,2022-08-26,AAPL230421C00050000,1,117.349998,AAPL,238
2,2022-08-29,AAPL230421C00050000,1,112.599998,AAPL,235
3,2022-09-01,AAPL230421C00050000,1,107.599998,AAPL,232
4,2022-09-02,AAPL230421C00050000,1,106.250000,AAPL,231


## Append the Strike Prices to the Dataframe

In [22]:
# Extract the strike price from the ticker
strikes = options_df[['contractSymbol', 'strike', 'impliedVolatility']]
# Join the strikes to the main dataframe
df_final = df_final.merge(strikes, how='left', left_on='Ticker', right_on='contractSymbol')
# Drop the 'contractSymbol' column
df_final = df_final.drop(columns=['contractSymbol'])
df_final.head()

,Date,Ticker,Call,Close,Stock Ticker,Days to Expiration,strike,impliedVolatility
0,2022-08-25,AAPL230421C00050000,1,119.500000,AAPL,239,50.0,4.187505
1,2022-08-26,AAPL230421C00050000,1,117.349998,AAPL,238,50.0,4.187505
2,2022-08-29,AAPL230421C00050000,1,112.599998,AAPL,235,50.0,4.187505
3,2022-09-01,AAPL230421C00050000,1,107.599998,AAPL,232,50.0,4.187505
4,2022-09-02,AAPL230421C00050000,1,106.250000,AAPL,231,50.0,4.187505


## Create a Dataframe with Daily Stock Prices

In [23]:
# Create a new dataframe
stocks_new = stocks_df

# Rename the 'Date' column to 'date_original'
stocks_new = stocks_new.rename(columns={'Date': 'date_original'})

# Get the first 10 characters from the 'date_original' column
# and put them in a new column called 'Date'
stocks_new['Date'] = stocks_new['date_original'].str[:10]

# Drop the date_original column
stocks_new = stocks_new.drop(columns=['date_original'])

# Create a unique identifier column
stocks_new['ID'] = stocks_new['Ticker'] + '-' + stocks_new['Date']

# Reorder the columns
stocks_new = stocks_new[['ID', 'Date', 'Ticker', 'Close']]

stocks_new.head()

,ID,Date,Ticker,Close
0,AAPL-2020-01-02,2020-01-02,AAPL,73.449409
1,AAPL-2020-01-03,2020-01-03,AAPL,72.735321
2,AAPL-2020-01-06,2020-01-06,AAPL,73.314880
3,AAPL-2020-01-07,2020-01-07,AAPL,72.970093
4,AAPL-2020-01-08,2020-01-08,AAPL,74.143913


## Create a Dataframe with Dividend Yields

In [24]:
# Create a new dataframe
dividends_new = dividends_df

# Rename the 'Date' column to 'date_original'
dividends_new = dividends_new.rename(columns={'Unnamed: 0': 'date_original'})

# Get the first 10 characters from the 'date_original' column
# and put them in a new column called 'Date'
dividends_new['Date'] = dividends_new['date_original'].str[:10]

# Drop the date_original column
dividends_new = dividends_new.drop(columns=['date_original'])

# Create a unique identifier column
dividends_new['ID'] = dividends_new['Ticker'] + '-' + dividends_new['Date']

# Reorder the columns
dividends_new = dividends_new[['ID', 'Date', 'Ticker', 'Dividends']]

dividends_new.head()

,ID,Date,Ticker,Dividends
0,AAPL-1987-05-11,1987-05-11,AAPL,0.000536
1,AAPL-1987-08-10,1987-08-10,AAPL,0.000536
2,AAPL-1987-11-17,1987-11-17,AAPL,0.000714
3,AAPL-1988-02-12,1988-02-12,AAPL,0.000714
4,AAPL-1988-05-16,1988-05-16,AAPL,0.000714


## Preview the Rates Dataframe

In [25]:
# Preview Rate Df
rate_df.head()

,Date,rate
0,2020-01-02,1.882
1,2020-01-03,1.788
2,2020-01-06,1.811
3,2020-01-07,1.827
4,2020-01-08,1.874


## Join the Stock Prices, Dividends, and Rate Dataframes

In [26]:
# Full Join the Stocks and Dividends Dataframes
stocks_dividends = pd.merge(stocks_new, dividends_new, how='outer', on=['ID'])

# Make a new Date column
stocks_dividends['Date'] = stocks_dividends['Date_x'].fillna(stocks_dividends['Date_y'])

# Make a new Ticker column
stocks_dividends['Ticker'] = stocks_dividends['Ticker_x'].fillna(stocks_dividends['Ticker_y'])

# Drop the 'Date_x' and 'Date_y' columns
stocks_dividends = stocks_dividends.drop(columns=['Date_x', 'Date_y'])
# Drop the 'Ticker_x' and 'Ticker_y' columns
stocks_dividends = stocks_dividends.drop(columns=['Ticker_x', 'Ticker_y'])

# Reset Index
stocks_dividends = stocks_dividends.reset_index()
# Drop the 'index' column
stocks_dividends = stocks_dividends.drop(columns=['index'])

# Reorder the columns
stocks_dividends = stocks_dividends[['ID', 'Date', 'Ticker', 'Close', 'Dividends']]

# Sort by Ticker and then by Date
stocks_dividends = stocks_dividends.sort_values(by=['Ticker', 'Date'])

# Impute Dividends
stocks_dividends['Dividends'] = stocks_dividends.groupby('Ticker')['Dividends'].transform(lambda x: x.fillna(method='ffill'))

# Drop NAs
stocks_dividends = stocks_dividends.dropna()

stocks_dividends

,ID,Date,Ticker,Close,Dividends
38869,AAP-2020-01-02,2020-01-02,AAP,148.484299,0.060
38870,AAP-2020-01-03,2020-01-03,AAP,148.493622,0.060
38871,AAP-2020-01-06,2020-01-06,AAP,146.053162,0.060
38872,AAP-2020-01-07,2020-01-07,AAP,144.320663,0.060
38873,AAP-2020-01-08,2020-01-08,AAP,142.662659,0.060
...,...,...,...,...,...
215933,ZTS-2023-04-10,2023-04-10,ZTS,170.869995,0.375
215934,ZTS-2023-04-11,2023-04-11,ZTS,171.210007,0.375
215935,ZTS-2023-04-12,2023-04-12,ZTS,171.449997,0.375
215936,ZTS-2023-04-13,2023-04-13,ZTS,173.539993,0.375


In [27]:
# Left Join with the stocks_dividends dataframe with the rate_df dataframe
stocks_dividends_rate = pd.merge(stocks_dividends, rate_df, how='left', on=['Date'])

stocks_dividends_rate

,ID,Date,Ticker,Close,Dividends,rate
0,AAP-2020-01-02,2020-01-02,AAP,148.484299,0.060,1.882
1,AAP-2020-01-03,2020-01-03,AAP,148.493622,0.060,1.788
2,AAP-2020-01-06,2020-01-06,AAP,146.053162,0.060,1.811
3,AAP-2020-01-07,2020-01-07,AAP,144.320663,0.060,1.827
4,AAP-2020-01-08,2020-01-08,AAP,142.662659,0.060,1.874
...,...,...,...,...,...,...
264926,ZTS-2023-04-10,2023-04-10,ZTS,170.869995,0.375,3.415
264927,ZTS-2023-04-11,2023-04-11,ZTS,171.210007,0.375,3.434
264928,ZTS-2023-04-12,2023-04-12,ZTS,171.449997,0.375,3.421
264929,ZTS-2023-04-13,2023-04-13,ZTS,173.539993,0.375,3.452


# Create an 'ID' column to uniquely identify each row

In [28]:
# Create a ID column for df_final
df_final['ID'] = df_final['Stock Ticker'] + '-' + df_final['Date'].astype(str)
# Reorder the columns
df_final = df_final[['ID', 'Date', 'Ticker', 'Call', 'Close', 'Stock Ticker',
                     'Days to Expiration', 'strike', 'impliedVolatility']]
df_final.head()

,ID,Date,Ticker,Call,Close,Stock Ticker,Days to Expiration,strike,impliedVolatility
0,AAPL-2022-08-25,2022-08-25,AAPL230421C00050000,1,119.500000,AAPL,239,50.0,4.187505
1,AAPL-2022-08-26,2022-08-26,AAPL230421C00050000,1,117.349998,AAPL,238,50.0,4.187505
2,AAPL-2022-08-29,2022-08-29,AAPL230421C00050000,1,112.599998,AAPL,235,50.0,4.187505
3,AAPL-2022-09-01,2022-09-01,AAPL230421C00050000,1,107.599998,AAPL,232,50.0,4.187505
4,AAPL-2022-09-02,2022-09-02,AAPL230421C00050000,1,106.250000,AAPL,231,50.0,4.187505


# Create new columns for each Observations Past 30 days of Underlying Data

In [29]:
# Create 20 new columns for the current day and 20 days before 'Date'
# i.e. Price-0, Price-1, Price-2, Price-3, etc.
for i in range(0, 31):
    df_final['Price-' + str(i)] = pd.to_datetime(df_final['Date']) - pd.DateOffset(days=i)
    df_final['Price-' + str(i)] = df_final['Price-' + str(i)].astype(str)
    df_final['Price-' + str(i)] = df_final['Price-' + str(i)].str[:10]

# Create 20 new columns for the current day and 20 days before 'Date'
# i.e. rate-0, rate-1, rate-2, rate-3, etc.
for i in range(0, 31):
    df_final['rate-' + str(i)] = pd.to_datetime(df_final['Date']) - pd.DateOffset(days=i)
    df_final['rate-' + str(i)] = df_final['rate-' + str(i)].astype(str)
    df_final['rate-' + str(i)] = df_final['rate-' + str(i)].str[:10]

# Create 20 new columns for the current day and 20 days before 'Date'
# i.e. Dividends-0, Dividends-1, Dividends-2, Dividends-3, etc.
for i in range(0, 31):
    df_final['Dividends-' + str(i)] = pd.to_datetime(df_final['Date']) - pd.DateOffset(days=i)
    df_final['Dividends-' + str(i)] = df_final['Dividends-' + str(i)].astype(str)
    df_final['Dividends-' + str(i)] = df_final['Dividends-' + str(i)].str[:10]

df_final.head()

,ID,Date,Ticker,Call,Close,Stock Ticker,Days to Expiration,strike,impliedVolatility,Price-0,...,Dividends-21,Dividends-22,Dividends-23,Dividends-24,Dividends-25,Dividends-26,Dividends-27,Dividends-28,Dividends-29,Dividends-30
0,AAPL-2022-08-25,2022-08-25,AAPL230421C00050000,1,119.500000,AAPL,239,50.0,4.187505,2022-08-25,...,2022-08-04,2022-08-03,2022-08-02,2022-08-01,2022-07-31,2022-07-30,2022-07-29,2022-07-28,2022-07-27,2022-07-26
1,AAPL-2022-08-26,2022-08-26,AAPL230421C00050000,1,117.349998,AAPL,238,50.0,4.187505,2022-08-26,...,2022-08-05,2022-08-04,2022-08-03,2022-08-02,2022-08-01,2022-07-31,2022-07-30,2022-07-29,2022-07-28,2022-07-27
2,AAPL-2022-08-29,2022-08-29,AAPL230421C00050000,1,112.599998,AAPL,235,50.0,4.187505,2022-08-29,...,2022-08-08,2022-08-07,2022-08-06,2022-08-05,2022-08-04,2022-08-03,2022-08-02,2022-08-01,2022-07-31,2022-07-30
3,AAPL-2022-09-01,2022-09-01,AAPL230421C00050000,1,107.599998,AAPL,232,50.0,4.187505,2022-09-01,...,2022-08-11,2022-08-10,2022-08-09,2022-08-08,2022-08-07,2022-08-06,2022-08-05,2022-08-04,2022-08-03,2022-08-02
4,AAPL-2022-09-02,2022-09-02,AAPL230421C00050000,1,106.250000,AAPL,231,50.0,4.187505,2022-09-02,...,2022-08-12,2022-08-11,2022-08-10,2022-08-09,2022-08-08,2022-08-07,2022-08-06,2022-08-05,2022-08-04,2022-08-03


## Append the Stock Prices, Dividends, and Rate Data to its Corresponding Row

In [30]:
# First, convert the 'Date' column to datetime in both DataFrames
stocks_dividends_rate['Date'] = pd.to_datetime(stocks_dividends_rate['Date'])
df_final['Date'] = pd.to_datetime(df_final['Date'])

# Loop through the range and merge the data for each 'Price-' column
for i in tqdm(range(0, 31)):
    date_col = f'Price-{i}'
    temp_df = stocks_dividends_rate[['Date', 'Ticker', 'Close']].copy()
    temp_df['Date'] = temp_df['Date'] + pd.DateOffset(days=i)
    
    # Merge the data and rename the 'Close' column to match the 'Price-' column name
    df_final = df_final.merge(temp_df, left_on=['Date', 'Stock Ticker'], right_on=['Date', 'Ticker'], how='left', suffixes=('', f'_{date_col}'))
    df_final.rename(columns={'Close' + f'_{date_col}': date_col}, inplace=True)
    df_final.drop(columns=['Ticker' + f'_{date_col}'], inplace=True)

# Create a new dataframe with the last 31 columns
df_final_2 = df_final.iloc[:, -31:]

# Drop all the columns that start with 'P'
df_final = df_final.filter(regex='^(?!P)')

# Concatenate the two dataframes
df_final = pd.concat([df_final, df_final_2], axis=1)

# Preview the columns that start with 'Price-'
df_final.filter(regex='Price-').head()

100%|██████████| 31/31 [02:46<00:00,  5.36s/it]


,Price-0,Price-1,Price-2,Price-3,Price-4,Price-5,Price-6,Price-7,Price-8,Price-9,...,Price-21,Price-22,Price-23,Price-24,Price-25,Price-26,Price-27,Price-28,Price-29,Price-30
0,169.489624,166.997574,166.698517,167.037445,NaN,NaN,170.974884,173.596527,173.995270,172.480103,...,165.053772,165.372330,159.280212,160.773376,NaN,NaN,161.768829,156.632355,156.074905,150.908585
1,163.100006,169.489624,166.997574,166.698517,167.037445,NaN,NaN,170.974884,173.596527,173.995270,...,164.824509,165.053772,165.372330,159.280212,160.773376,NaN,NaN,161.768829,156.632355,156.074905
2,160.867111,NaN,NaN,163.100006,169.489624,166.997574,166.698517,167.037445,NaN,NaN,...,164.346024,NaN,NaN,164.824509,165.053772,165.37233,159.280212,160.773376,NaN,NaN
3,157.457993,156.720337,158.404968,160.867111,NaN,NaN,163.100006,169.489624,166.997574,166.698517,...,167.954529,168.702148,164.395874,164.346024,NaN,NaN,164.824509,165.053772,165.372330,159.280212
4,155.314819,157.457993,156.720337,158.404968,160.867111,NaN,NaN,163.100006,169.489624,166.997574,...,171.553055,167.954529,168.702148,164.395874,164.346024,NaN,NaN,164.824509,165.053772,165.372330


In [31]:
# Do the same thing but with Dividends
for i in tqdm(range(0, 31)):
    date_col = f'Dividends-{i}'
    temp_df = stocks_dividends_rate[['Date', 'Ticker', 'Dividends']].copy()
    temp_df['Date'] = temp_df['Date'] + pd.DateOffset(days=i)
    
    # Merge the data and rename the 'Close' column to match the 'Price-' column name
    df_final = df_final.merge(temp_df, left_on=['Date', 'Stock Ticker'], right_on=['Date', 'Ticker'], how='left', suffixes=('', f'_{date_col}'))
    df_final.rename(columns={'Dividends' + f'_{date_col}': date_col}, inplace=True)
    df_final.drop(columns=['Ticker' + f'_{date_col}'], inplace=True)

# Create a new dataframe with the last 31 columns
df_final_2 = df_final.iloc[:, -31:]

# Drop all the columns that start with 'Dividends'
df_final = df_final.filter(regex='^(?!Dividends)')

# Concatenate the two dataframes
df_final = pd.concat([df_final, df_final_2], axis=1)

# Preview the columns that start with 'Dividends-'
df_final.filter(regex='Dividends-').head()

100%|██████████| 31/31 [01:19<00:00,  2.58s/it]


,Dividends-1,Dividends-2,Dividends-3,Dividends-4,Dividends-5,Dividends-6,Dividends-7,Dividends-8,Dividends-9,Dividends-10,...,Dividends-21,Dividends-22,Dividends-23,Dividends-24,Dividends-25,Dividends-26,Dividends-27,Dividends-28,Dividends-29,Dividends-30
0,0.23,0.23,0.23,NaN,NaN,0.23,0.23,0.23,0.23,0.23,...,0.23,0.23,0.23,0.23,NaN,NaN,0.23,0.23,0.23,0.23
1,0.23,0.23,0.23,0.23,NaN,NaN,0.23,0.23,0.23,0.23,...,0.23,0.23,0.23,0.23,0.23,NaN,NaN,0.23,0.23,0.23
2,NaN,NaN,0.23,0.23,0.23,0.23,0.23,NaN,NaN,0.23,...,0.23,NaN,NaN,0.23,0.23,0.23,0.23,0.23,NaN,NaN
3,0.23,0.23,0.23,NaN,NaN,0.23,0.23,0.23,0.23,0.23,...,0.23,0.23,0.23,0.23,NaN,NaN,0.23,0.23,0.23,0.23
4,0.23,0.23,0.23,0.23,NaN,NaN,0.23,0.23,0.23,0.23,...,0.23,0.23,0.23,0.23,0.23,NaN,NaN,0.23,0.23,0.23


In [32]:
# Convert the stocks_dividends_rate DataFrame to a dictionary
# with the 'Date' as the key and 'rate' as the value
stocks_dividends_rate_dict = stocks_dividends_rate.set_index('Date')['rate'].to_dict()

# Create a function that queries the rate for a given date
# from the stocks_dividends_rate_dict
def get_rate(date):
    return stocks_dividends_rate_dict.get(date, np.nan)

# Replace the values in the 'rate-' columns with the
# values from the rates_list using the apply function
for i in tqdm(range(0, 31)):
    df_final[f'rate-{i}'] = df_final['Date'].apply(get_rate)

# Preview the first 5 rows of the 'rate-' columns
df_final.filter(regex='rate-').head()

100%|██████████| 31/31 [00:30<00:00,  1.02it/s]


,rate-0,rate-1,rate-2,rate-3,rate-4,rate-5,rate-6,rate-7,rate-8,rate-9,...,rate-21,rate-22,rate-23,rate-24,rate-25,rate-26,rate-27,rate-28,rate-29,rate-30
0,3.026,3.026,3.026,3.026,3.026,3.026,3.026,3.026,3.026,3.026,...,3.026,3.026,3.026,3.026,3.026,3.026,3.026,3.026,3.026,3.026
1,3.035,3.035,3.035,3.035,3.035,3.035,3.035,3.035,3.035,3.035,...,3.035,3.035,3.035,3.035,3.035,3.035,3.035,3.035,3.035,3.035
2,3.110,3.110,3.110,3.110,3.110,3.110,3.110,3.110,3.110,3.110,...,3.110,3.110,3.110,3.110,3.110,3.110,3.110,3.110,3.110,3.110
3,3.265,3.265,3.265,3.265,3.265,3.265,3.265,3.265,3.265,3.265,...,3.265,3.265,3.265,3.265,3.265,3.265,3.265,3.265,3.265,3.265
4,3.193,3.193,3.193,3.193,3.193,3.193,3.193,3.193,3.193,3.193,...,3.193,3.193,3.193,3.193,3.193,3.193,3.193,3.193,3.193,3.193


## Drop the Columns that Represent the Days where the Market was Closed

In [33]:
# Eliminate the columns for the days the market was closed
cols_to_drop = ['Price-1', 'Price-2', 'Price-3', 'Price-8', 'Price-9', 
                'Price-15', 'Price-16', 'Price-22', 'Price-23', 
                'Price-29', 'Price-30',
                'Dividends-1', 'Dividends-2', 'Dividends-3',
                'Dividends-8', 'Dividends-9', 'Dividends-15', 'Dividends-16',
                'Dividends-22', 'Dividends-23', 'Dividends-29', 'Dividends-30',
                'rate-1', 'rate-2', 'rate-3', 'rate-8', 'rate-9', 'rate-15',
                'rate-16', 'rate-22', 'rate-23', 'rate-29', 'rate-30']
# Drop the columns
df_final = df_final.drop(cols_to_drop, axis=1)

## Impute the Missing Values using Backward Fill and Forward Fill

In [34]:
# Create a Dataframe with only the columns that start with the word Price
price_cols = [col for col in df_final.columns if col.startswith('Price')]
prices_df = df_final[price_cols]
prices_trans = prices_df.transpose()
# Implement backward fill to fill in the missing values
prices_back = prices_trans.fillna(method='bfill', axis=0)
# Implement forward fill to fill in the missing values
prices_for = prices_back.fillna(method='ffill', axis=0)
# Transpose the Dataframe
prices_new = prices_for.transpose()


# Create a Dataframe with only the columns that start with the
# word Dividends
dividends_cols = [col for col in df_final.columns
                  if col.startswith('Dividends')]
divs_df = df_final[dividends_cols]
divs_trans = divs_df.transpose()
# Implement backward fill to fill in the missing values
divs_back = divs_trans.fillna(method='bfill', axis=0)
# Implement forward fill to fill in the missing values
divs_for = divs_back.fillna(method='ffill', axis=0)
# Transpose the Dataframe
divs_new = divs_for.transpose()

# Create a Dataframe with only the columns that start with the word rate
rate_cols = [col for col in df_final.columns if col.startswith('rate')]
rates_df = df_final[rate_cols]
rates_trans =  rates_df.transpose()
# Implement backward fill to fill in the missing values 
rates_back = rates_trans.fillna(method='bfill', axis=0)
# Implement forward fill to fill in the missing values
rates_for = rates_back.fillna(method='ffill', axis=0)
# Transpose the Dataframe
rates_new = rates_for.transpose()

# Replace the old columns with the new ones
for i in range(0, 31):
    try:
        date_col = f'Price-{i}'
        df_final[date_col] = prices_new[date_col]
        date_col = f'Dividends-{i}'
        df_final[date_col] = divs_new[date_col]
        date_col = f'rate-{i}'
        df_final[date_col] = rates_new[date_col]
    except:
        pass

# Preview the first 5 rows of the Dataframe
df_final.head()

,ID,Date,Ticker,Call,Close,Stock Ticker,Days to Expiration,strike,impliedVolatility,rate-0,...,Dividends-17,Dividends-18,Dividends-19,Dividends-20,Dividends-21,Dividends-24,Dividends-25,Dividends-26,Dividends-27,Dividends-28
0,AAPL-2022-08-25,2022-08-25,AAPL230421C00050000,1,119.500000,AAPL,239,50.0,4.187505,3.026,...,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23
1,AAPL-2022-08-26,2022-08-26,AAPL230421C00050000,1,117.349998,AAPL,238,50.0,4.187505,3.035,...,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23
2,AAPL-2022-08-29,2022-08-29,AAPL230421C00050000,1,112.599998,AAPL,235,50.0,4.187505,3.110,...,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23
3,AAPL-2022-09-01,2022-09-01,AAPL230421C00050000,1,107.599998,AAPL,232,50.0,4.187505,3.265,...,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23
4,AAPL-2022-09-02,2022-09-02,AAPL230421C00050000,1,106.250000,AAPL,231,50.0,4.187505,3.193,...,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23


## Drop the Rows that Still Contain Missing Values

In [35]:
# Drop Rows with NaN values
df_final = df_final.dropna()

In [36]:
# Preview Data
df_final.head()

,ID,Date,Ticker,Call,Close,Stock Ticker,Days to Expiration,strike,impliedVolatility,rate-0,...,Dividends-17,Dividends-18,Dividends-19,Dividends-20,Dividends-21,Dividends-24,Dividends-25,Dividends-26,Dividends-27,Dividends-28
0,AAPL-2022-08-25,2022-08-25,AAPL230421C00050000,1,119.500000,AAPL,239,50.0,4.187505,3.026,...,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23
1,AAPL-2022-08-26,2022-08-26,AAPL230421C00050000,1,117.349998,AAPL,238,50.0,4.187505,3.035,...,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23
2,AAPL-2022-08-29,2022-08-29,AAPL230421C00050000,1,112.599998,AAPL,235,50.0,4.187505,3.110,...,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23
3,AAPL-2022-09-01,2022-09-01,AAPL230421C00050000,1,107.599998,AAPL,232,50.0,4.187505,3.265,...,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23
4,AAPL-2022-09-02,2022-09-02,AAPL230421C00050000,1,106.250000,AAPL,231,50.0,4.187505,3.193,...,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23,0.23


# Reorder Columns

In [5]:
# Rename a Misnamed Column
df_final.rename(columns={'Dividends': 'Dividends-0'}, inplace=True)
# Generate a list of the columns in the order we want them
col_list = ['ID', 'Date', 'Ticker', 'Call', 'Close', 'Stock Ticker',
       'Days to Expiration', 'strike', 'impliedVolatility', 'Price-0',
       'Price-4', 'Price-5', 'Price-6', 'Price-7', 'Price-10', 'Price-11',
       'Price-12', 'Price-13', 'Price-14', 'Price-17', 'Price-18', 'Price-19',
       'Price-20', 'Price-21', 'Price-24', 'Price-25', 'Price-26', 'Price-27',
       'Price-28', 'rate-0', 'rate-4', 'rate-5', 'rate-6', 'rate-7', 'rate-10',
       'rate-11', 'rate-12', 'rate-13', 'rate-14', 'rate-17', 'rate-18',
       'rate-19', 'rate-20', 'rate-21', 'rate-24', 'rate-25', 'rate-26',
       'rate-27', 'rate-28', 'Dividends-0', 'Dividends-4', 'Dividends-5',
       'Dividends-6', 'Dividends-7', 'Dividends-10', 'Dividends-11',
       'Dividends-12', 'Dividends-13', 'Dividends-14', 'Dividends-17',
       'Dividends-18', 'Dividends-19', 'Dividends-20', 'Dividends-21',
       'Dividends-24', 'Dividends-25', 'Dividends-26', 'Dividends-27',
       'Dividends-28']
# Reorder the columns
df_final = df_final[col_list]

## Export the Final DataFrame to a Parquet File

In [5]:
# Export the dataframe to a Parquet File
df_final.to_parquet('02_Clean_Data\/df_final.parquet')